In [4]:
import random
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import matplotlib
matplotlib.use('agg')  #可以减少内存的使用
import matplotlib.pyplot as plt
import collections
from numpy import cos, sin
import copy
import pandas as pd
import math

random.seed(0)
torch.manual_seed(0)
np.random.seed(0)
torch.cuda.manual_seed_all(0)

xuhao = 14

###超参数调试区  
#基本不用动的参数
gamma = 0.99
tau = 0.002  # 软更新参数
buffer_size = 24000  # 40000
minimal_size = 4000  # 1000
batch_size = 128 # 128

sigma = 0.01  # 高斯噪声标准差

action_bound = 0.245 #动作范围
noise = 1
noise_1_bound = noise
#需要调参区   暴力破解
actor_lr = xuhao * 1e-4#学习率
critic_lr = xuhao * 1e-3
episodes = 4000  #回合数
Tmax = 40  #仿真时间
steps = 400  #步数
state_limit_episode = minimal_size/steps  #神经网络开始学习需要的回合数

def reward_function(error,x1dot,x1c,x1): #奖励函数
    bound = 1
    if x1 <x1c - bound and x1dot>0 :
        a = - 20*np.sqrt((error/20)**2)  + 1    
    elif x1> x1c+ bound and x1dot<0:
        a = - 20*np.sqrt((error/20)**2)  + 1
    elif x1c- bound <= x1 <=x1c + bound:
        a =  10 *  np.absolute(1 - np.absolute(error)/bound)
    else:
        a =  - 20*np.sqrt((error/20)**2)

    reward = a 

    return reward

class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = collections.deque(maxlen=capacity)

    def add(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        transitions = random.sample(self.buffer, batch_size)  # 从指定的序列中，随机的截取指定长度的片段
        state, action, reward, next_state, done = zip(*transitions)
        return np.array(state), action, reward, np.array(next_state), done

    def size(self):
        return len(self.buffer)

def soft_update(net, target_net):  # 采用软更新
    for param_target, param in zip(target_net.parameters(), net.parameters()):
        param_target.data.copy_(param_target.data * (1.0 - tau) + param.data * tau)

class Net(nn.Module):
    def __init__(self, num_in, num_out, hidden_dim, fn=lambda x: (torch.tanh(x))):  # 最早是1
        super(Net, self).__init__()
        self.fc1 = nn.Linear(num_in, 64)
        self.fc2 = nn.Linear(64, hidden_dim)
        self.fc22 = nn.Linear(hidden_dim, 64)
        self.fc3 = nn.Linear(64, num_out)
        self.activation = nn.ReLU()
        self.fn = fn

    def forward(self, x):
        x = self.activation(self.fc1(x))
        # print(x)
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc22(x))
        x = self.fc3(x)
        x = self.fn(x)
        return x

actor = Net(3, 1, 64).to("cuda")
target_actor = Net(3, 1, 64).to("cuda")
critic_1 = Net(4,1,128,fn = lambda x:x).to("cuda")
target_critic_1 = Net(4,1,128,fn = lambda x:x).to("cuda")
critic_2 = Net(4,1,128,fn = lambda x:x).to("cuda")
target_critic_2 = Net(4,1,128,fn = lambda x:x).to("cuda")
target_actor.load_state_dict(actor.state_dict())
target_critic_1.load_state_dict(critic_1.state_dict())
target_critic_2.load_state_dict(critic_2.state_dict())

def update(tran_dict,episode):
    states = torch.tensor(tran_dict['states'], dtype=torch.float).to("cuda")
    actions = torch.tensor(tran_dict['actions'], dtype=torch.float).view(-1, 1).to("cuda")
    next_states = torch.tensor(tran_dict['next_states'], dtype=torch.float).to("cuda")
    dones = torch.tensor(tran_dict['dones'], dtype=torch.float).view(-1, 1).to("cuda")
    rewards = torch.tensor(tran_dict['rewards'], dtype=torch.float).view(-1, 1).to("cuda")
    #下一时刻做出下一时刻动作的q值
    global noise_1_bound
    
    next_q_values_min = None

    #目标策略平滑，在目标策略的输出动作中加入噪声，以此来平滑q函数的估值，避免过拟合。
    #在此处加入高斯噪音并clip截断
    noise_1 = sigma * np.random.randn() * noise_1_bound  #标准正态分布
    
    #虽然要添加噪音来平滑，但是还是要逐渐减小为好
    noise_1_bound = noise_1_bound * 0.99
    
    #clip将高斯噪音进行限幅
    noise_1 = np.clip(noise_1,-sigma*2*noise_1_bound,sigma*2*noise_1_bound)
    
    #将加入噪音的动作进行限幅
    target_action = torch.clamp(target_actor(next_states) + torch.tensor(noise_1,dtype=torch.float).to("cuda"),-1,1)   #clamp限幅函数
    #限制最终的target动作在-1到1之间//////

    next_q_values_1 = target_critic_1(torch.cat([next_states, target_action], dim=1))
    next_q_values_2 = target_critic_2(torch.cat([next_states, target_action], dim=1))
    #此处也未参与反向传播
    
    #计算目标值时，利用二者间的较小值来进行计算
    if next_q_values_1[0] > next_q_values_2[0]:
        next_q_values_min = next_q_values_2
    else:
        next_q_values_min = next_q_values_1  

    #这一时刻做出这一动作的目标q值  #也要分开不然报错
    q_targets = rewards + gamma * next_q_values_min * (1 - dones)
    #两个价值网络都进行损失函数的反向传播
    #误差函数要分开写，不然会报错
    #critic_1网络的更新
    critic_loss_1 = torch.mean(F.mse_loss(critic_1(torch.cat([states, actions], dim=1)), q_targets))
    critic_optimizer_1.zero_grad()
    critic_loss_1.backward(retain_graph=True)  #要加上这个才行
    critic_optimizer_1.step()
    #critic_2网络的更新
    critic_loss_2 = torch.mean(F.mse_loss(critic_2(torch.cat([states, actions], dim=1)), q_targets))
    critic_optimizer_2.zero_grad()
    critic_loss_2.backward()
    critic_optimizer_2.step()
    
    #每隔d个回合再进行actor网络以及所有目标网络的更新
    #%为求episode的余数，可以理解为每隔2个回合过呢更新一次
    if episode % 2 == 0 :  #如果episode恰好可以被2整除，也就是偶数回合
        #actor网络的损失函数
        #ps：伪代码中只是利用critic_1网络来进行actor网络的学习，应该影响不大
        actor_loss = -torch.mean(critic_1(torch.cat([states,actor(states)],dim=1)))
        actor_optimizer.zero_grad()
        actor_loss.backward()
        actor_optimizer.step()     
        #软更新目标网络
        soft_update(actor,target_actor)
        soft_update(critic_1,target_critic_1)
        soft_update(critic_2,target_critic_2)

class env:  #模型环境
    def __init__(self):
        #self.V = 20
        self.g = 9.8
        self.h = 20
        self.R = 2.5
        self.Cx0 = 0.1
        self.dt = Tmax/steps  #单位时间
        self.omega_n = 3.66
        self.a0 = -0.3379
        self.a1 = 6.8762
        self.a2 = 1.487
        self.rhow = 998
        self.x1c = 6
        self.Dn = self.R*2  ##等效空化器直径
        self.p0 = 101000  ##大气压强
        self.pc = 3500    ##饱和蒸汽压 
    def reset(self,episode):
        self.t = 0
        
        self.V =  3 * np.sin(np.pi*self.t/5) + 27
        #更新
        self.x1 = 0
        self.x2 = 0
        
        self.LcavReal = self.x1
        self.LcavRealdot = self.x2        
        
        self.done = False
#         #
#         self.Fr = self.V/np.sqrt(self.g*self.h)  #定值
#         self.Sigmac = self.Fr**(-4) + self.Fr**(-2) * (self.Fr**(-4) + 0.073/(4*self.Qin))**2  #变
#         self.cd = self.Cx0 * (1 + self.Sigmac) #变
#         self.LcavSteady = 2 * self.R * np.sqrt(self.cd * np.log(1/self.Sigmac))/self.Sigmac #变        
        
        self.error = self.x1c - self.x1
        #self.x2,self.Sigmac,self.cd,self.LcavSteady,self.x1,self.error
        return [self.x1,self.error,self.V]

    def step(self, action ,episode):
        
        self.Qin= action[0]
        self.cq = self.Qin / (self.V*self.Dn**2)
        self.Fr = self.V/np.sqrt(self.g*self.h)  #定值
        self.k = 16/(0.015*np.pi*self.Cx0)
        self.Sigmav = (self.g*self.h*self.rhow+self.p0-self.pc)/(0.5*self.rhow*self.V**2)
        self.Sigmac = 2*self.Sigmav/(1+np.sqrt(1+self.k*self.Sigmav*self.cq))
        self.cd = self.Cx0 * (1 + self.Sigmac) #变
        self.LcavSteady = 2 * self.R * np.sqrt(self.cd * np.log(1/self.Sigmac))/self.Sigmac #变
        
        #四阶龙格库塔积分
        kx = np.tanh(self.a1 * self.LcavRealdot) * self.a0 + self.a2
        
        x1dot = self.x2 
        x2dot = -(self.omega_n)**2 * self.x1 - 2 * kx * self.omega_n * self.x2 + (self.omega_n)**2 * self.LcavSteady
        k1 =   [self.dt*x1dot,self.dt*x2dot]
        
        x1dot = self.x2 + 0.5 * k1[1]
        x2dot = -(self.omega_n)**2 * (self.x1 + 0.5 * k1[0]) -2 * kx * self.omega_n * (self.x2 + 0.5 * k1[1]) + (self.omega_n)**2 * self.LcavSteady
        k2 =  [self.dt*x1dot,self.dt*x2dot]
        
        x1dot = self.x2 + 0.5 * k2[1]
        x2dot = -(self.omega_n)**2 * (self.x1 + 0.5 * k2[0]) -2 * kx * self.omega_n * (self.x2 + 0.5 * k2[1]) + (self.omega_n)**2 * self.LcavSteady
        k3 = [self.dt*x1dot,self.dt*x2dot]
        
        x1dot = self.x2 + k3[1]
        x2dot = -(self.omega_n)**2 * (self.x1 + k3[0]) -2 * kx * self.omega_n * (self.x2 +  k3[1]) + (self.omega_n)**2 * self.LcavSteady
        k4 =[self.dt*x1dot,self.dt*x2dot]
        
        self.x1 = self.x1 + (k1[0] + 2* k2[0] + 2 * k3[0] + k4[0])/6
        self.x2 = self.x2 + (k1[1] + 2* k2[1] + 2 * k3[1] + k4[1])/6
        
        self.LcavReal = self.x1
        self.LcavRealdot = self.x2
        
        self.error = self.x1c - self.x1
        
        self.r = reward_function(self.error,self.x2,self.x1c,self.x1)
        
        self.t = self.t + self.dt
        
        self.V = 3 * np.sin(np.pi*self.t/5) + 27
        
        #判断是否超出状态空间
        if self.x1 > 20 or self.x1 < -5:
            
            self.done = True  # 真的出界
            #
        return [self.x1,self.error,self.V], self.r, self.done

gaochao = env()

#存储所有回合的奖励
return_list = []
#存储每个回合的动作、奖励、动作
state_buffer = []
reward_buffer = []
action_buffer = []
#经验池
replay_buffer = ReplayBuffer(buffer_size)
actor_optimizer = torch.optim.NAdam(actor.parameters(), lr=actor_lr)
critic_optimizer_1 = torch.optim.NAdam(critic_1.parameters(), lr=critic_lr)
critic_optimizer_2 = torch.optim.NAdam(critic_2.parameters(), lr=critic_lr)

for episode in range(episodes):
    noise = noise * 0.993  #噪音随着回合数的增大而逐渐减小
    episode_return = 0 #每回合开始时将回合奖励清零
    state = gaochao.reset(episode)  #环境给出初始参数
    done = False  #超出警告标志回归为未超出状态空间范围
    #将上个回合的动作、奖励、动作清除
    state_buffer.clear()
    reward_buffer.clear()
    action_buffer.clear()    
#     print(episode)  #输出是第几个回合
    for step in range(steps):
#         #复制状态，将复制的状态经过归一化后输入到actor网络中，并最终存入经验池中
#         print(state)
        st = copy.deepcopy(state) 
#         st[0] = st[0] / 20
#         st[1] = st[1] / 1
#         st[2] = st[2] / 0.2
#         st[3] = st[3] / 1.2
        st[0] = st[0] / 20
        st[1] = st[1] / 20
        st[2] = st[2] / 30
        #将归一化的状态量输入actor'网络中
        s = torch.tensor(st,dtype = torch.float).to("cuda") #更改格式只有tensor格式才能输入到神经网络中
        action = actor(s)
        
        action = action.tolist() #转换为list格式
        
        ac = copy.deepcopy(action)  #复制动作并进行归一化以便存入经验池
        
        action[0] = 0.255 + action[0] * action_bound + sigma * np.random.randn() * noise #将输出值化为动作并添加噪音
        
        if action[0] > 0.5:
            action[0] = 0.5
        if action[0] < 0.01:
            action[0] = 0.01
            
        action_buffer.append(action)  #存动作

        #对环境输入动作，输出下一时刻的状态、奖励、标志
        next_state, reward , done = gaochao.step(action,episode)
        
        if done == True : 
            break
        
        state_buffer.append(state)  #存状态
        reward_buffer.append(reward) #存奖励
        
        ns = copy.deepcopy(next_state)  #复制下一时刻的状态并归一化
#         ns[0] = ns[0] / 20
#         ns[1] = ns[1] / 1
#         ns[2] = ns[2] / 0.2
#         ns[3] = ns[3] / 1.2
        ns[0] = ns[0] / 20
        ns[1] = ns[1] / 20
        ns[2] = ns[2] / 30
        replay_buffer.add(st,ac,reward,ns,done)  #将状态/动作/奖励/下一时刻的状态/是否越界存入经验池
        
        state = next_state #更新状态
        episode_return += reward #加上这一步的奖励
        
        if replay_buffer.size() > minimal_size:  #如果经验池中的元组数超过神经网络开始学习所需要的元组数
            b_s , b_a , b_r , b_ns , b_d = replay_buffer.sample(batch_size) #返还指定批量的元组
            #设置字典方便学习
            transition_dict = {'states': b_s, 'actions': b_a, 'next_states': b_ns, 'rewards': b_r, 'dones': b_d}
            update(transition_dict,episode)  #神经网络进行更新
            
            
#     print(episode_return) #输出每回合的奖励
#     print("--------------") #分隔线有利于观察
    
    return_list.append(episode_return) #将本回合的奖励存入所有奖励的列表中
    

#    torch.save(actor.state_dict(),'D:/pjy/actor_net/{}.pth'.format(episode)) 
#存数据区
  #  state_buffer = np.array(state_buffer) #需要先将列表转化为矩阵格式才能调取其中的
    #存状态量的变化数据
    #self.alpha, self.v2, self.fu, self.Wz,self.T,self.L
#     data = pd.DataFrame(state_buffer[:,0])
#     data.to_csv('D:/pjy/state_data/{}alpha.csv'.format(episode))
#     data = pd.DataFrame(state_buffer[:,1])
#     data.to_csv('D:/pjy/state_data/{}v2.csv'.format(episode))  
#     data = pd.DataFrame(state_buffer[:,2])
#     data.to_csv('D:/pjy/state_data/{}fu.csv'.format(episode))    
#     data = pd.DataFrame(state_buffer[:,3])
#     data.to_csv('D:/pjy/state_data/{}Wz.csv'.format(episode))    
#     data = pd.DataFrame(state_buffer[:,4])
#     data.to_csv('D:/pjy/state_data/{}T.csv'.format(episode))    
#     data = pd.DataFrame(state_buffer[:,5])
#     data.to_csv('D:/pjy/state_data/{}L.csv'.format(episode))    

    #存动作量的变化数据
#     data = pd.DataFrame(action_buffer)
#     data.to_csv('D:/pjy/action_data/{}.csv'.format(episode))    
    
    
    #画图区
    state_buffer = np.array(state_buffer)
    u = np.arange(0,400,0.01) #目标值的x轴数值
    target = np.full(shape=(40000,1),fill_value = 6) #目标值的y轴数值  fill_value为目标值
    
    x1 = len(state_buffer)
    x1 = np.arange(x1) #state的x轴数值
    plt.figure() #新建一个画布
    
    plt.title('state %1.3f' % episode)  #标题
    plt.plot(x1 , state_buffer[:,0]) #绘状态变化的图
    plt.plot(u,target) #绘制目标值的直线
    
    state_buffer = state_buffer.tolist()
#     plt.savefig("./pjy{}/png/{}.png".format(xuhao,episode))  #将画好的图存到指定的位置上
# #     plt.show()

#     x2 = len(action_buffer)
#     x2 = np.arange(x2)
#     plt.figure()
#     plt.title('action %1.3f' %episode)
#     plt.plot(x2 , action_buffer)
#     plt.savefig("./pjy{}/action_data/{}.png".format(xuhao,episode))
    
#     plt.close("all")  #将所有的画布删去防止占用内存

    
#  833  841  957   1165  1385                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    









d:\tiannananna\venv\lib\site-packages\ipykernel_launcher.py:377: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
